In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import surprise
%matplotlib inline

In [2]:
ls notebooks/

gl-collaborative-filetering-model.ipynb  Zuzu-notebook.ipynb
guillaume-GL-kickoff.ipynb               zuzu_rawg_games.ipynb
mag_content_based.ipynb                  zuzu_rawg_user_games.ipynb
mr-gl-split-and-testing.ipynb            zuzu_rawg_users.ipynb
Rasha.ipynb


In [3]:
def load_data():
    user_game_df = pd.read_csv("raw_data/rawg_user_games.csv")
    user_game_df['purchase'] = 1
    user_game_df = user_game_df[user_game_df['user_rating']>0]
    return user_game_df

In [4]:
ratings_data = load_data()
ratings_data

/home/rasha/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3347: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,user_id,game_id,game_slug,game_name,user_rating,metacritic,rawg_rating,released,play_time,purchase
0,1,446900,high-rise-a-puzzle-cityscape,High Rise - A Puzzle Cityscape,4,NaN,0.00,2020-05-16,0,1
1,1,264830,twinfold,Twinfold,5,NaN,0.00,2018-12-05,0,1
2,1,58443,pocket-run-pool,Pocket-Run Pool,4,NaN,0.00,2018-05-24,0,1
4,1,21889,desert-fox-the-battle-of-el-alamein,Desert Fox: The Battle of El Alamein,4,NaN,0.00,2014-06-26,0,1
5,1,1190,imbroglio,Imbroglio,4,NaN,0.00,2016-05-18,0,1
...,...,...,...,...,...,...,...,...,...,...
255492,rasha27,5578,grid,Race Driver: Grid,5,87.0,3.67,2008-06-03,1,1
255493,rasha27,326292,fall-guys,Fall Guys: Ultimate Knockout,3,80.0,3.87,2020-08-04,4,1
255494,rasha27,2572,dirt-rally,DiRT Rally,5,86.0,3.74,2015-12-07,4,1
255495,rasha27,4003,grid-2,GRID 2,5,80.0,3.30,2013-05-27,4,1


In [5]:
metadata = ratings_data[['game_id','game_name', 'released', 'metacritic', 'rawg_rating']]
metadata

,game_id,game_name,released,metacritic,rawg_rating
0,446900,High Rise - A Puzzle Cityscape,2020-05-16,NaN,0.00
1,264830,Twinfold,2018-12-05,NaN,0.00
2,58443,Pocket-Run Pool,2018-05-24,NaN,0.00
4,21889,Desert Fox: The Battle of El Alamein,2014-06-26,NaN,0.00
5,1190,Imbroglio,2016-05-18,NaN,0.00
...,...,...,...,...,...
255492,5578,Race Driver: Grid,2008-06-03,87.0,3.67
255493,326292,Fall Guys: Ultimate Knockout,2020-08-04,80.0,3.87
255494,2572,DiRT Rally,2015-12-07,86.0,3.74
255495,4003,GRID 2,2013-05-27,80.0,3.30


In [6]:
from surprise import Dataset
from surprise import Reader
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_data[['user_id', 'game_id', 'user_rating']], reader)

In [7]:
from surprise import SVD
from surprise.model_selection import cross_validate
svd = SVD(verbose=True, n_epochs=10)
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.9786  0.9796  0.9719  0.9767  0.0034  
MAE (testset)     0.7370  0.7336  0.7279  0.7328  0.0038  
Fit time          0.50    0.48    0.50    0.49    0.01    
Test time         0.06    0.07    0.10    0.08    0.02    


{'test_rmse': array([0.97856016, 0.97955616, 0.97185806]),
 'test_mae': array([0.73697666, 0.7336002 , 0.72785316]),
 'fit_time': (0.5019075870513916, 0.4764535427093506, 0.4956169128417969),
 'test_time': (0.06232953071594238, 0.06778168678283691, 0.10102701187133789)}

In [9]:
trainset = data.build_full_trainset()
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


In [10]:
svd.predict(uid=10, iid=100)

Prediction(uid=10, iid=100, r_ui=None, est=3.6756172209753113, details={'was_impossible': False})

In [11]:
import difflib
import random

def get_game_id(game_name, metadata):
    """
    Gets the book ID for a book title based on the closest match in the metadata dataframe.
    """
    existing_titles = list(metadata['game_name'].values)
    closest_titles = difflib.get_close_matches(game_name, existing_titles)
    game_id = metadata[metadata['game_name'] == closest_titles[0]]['game_id'].values[0]
    return game_id

def get_game_info(game_id, metadata):
    """
    Returns some basic information about a book given the book id and the metadata dataframe.
    """
    game_info = metadata[metadata['game_id'] == game_id][['game_id','game_name', 'released', 'metacritic', 'rawg_rating']]
    
    return game_info.to_dict(orient='records')

def predict_review(user_id, game_name, model, metadata):
    """
    Predicts the purchase state (on a scale of 0-1) that a user would assign to a specific book. 
    """
    game_id = get_game_id(game_name, metadata)
    review_prediction = model.predict(uid=user_id, iid=game_id)
    return review_prediction.est

def generate_recommendation(user_id, model, metadata, thresh=4):
    
    """
    Generates a game recommendation for a user based on a rating threshold. Only
    games with a predicted rating at or above the threshold will be recommended
    """
    
    game_names = list(metadata['game_name'].values)
    random.shuffle(game_names)
    
    for game_name in game_names:
        rating = predict_review(user_id, game_name, model, metadata)
        if rating >= thresh:
            game_id = get_game_id(game_name, metadata)
            return get_game_info(game_id, metadata)

        

In [15]:
csv = generate_recommendation(1000, svd, metadata)


In [18]:
pd.DataFrame.from_dict(csv)

,game_id,game_name,released,metacritic,rawg_rating
0,58755,Devil May Cry 5,2019-03-08,88.0,4.24
1,58755,Devil May Cry 5,2019-03-08,88.0,4.24
2,58755,Devil May Cry 5,2019-03-08,88.0,4.24
3,58755,Devil May Cry 5,2019-03-08,88.0,4.24
4,58755,Devil May Cry 5,2019-03-08,88.0,4.24
5,58755,Devil May Cry 5,2019-03-08,88.0,4.24
6,58755,Devil May Cry 5,2019-03-08,88.0,4.24
7,58755,Devil May Cry 5,2019-03-08,88.0,4.24
8,58755,Devil May Cry 5,2019-03-08,88.0,4.24
9,58755,Devil May Cry 5,2019-03-08,88.0,4.24
